In [ ]:
#importamos las librerias
import os
import pandas as pd
import re

In [ ]:
#Vamos a instalar unas libreria, especificamente la libreria transformers de "huggingface"
!pip install transformers

     |████████████████████████████████| 1.3MB 6.2MB/s 
     |████████████████████████████████| 1.1MB 33.2MB/s 
     |████████████████████████████████| 2.9MB 37.7MB/s 
     |████████████████████████████████| 890kB 42.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=8d28344032779dd4e329348281549d2ec1680543188259447ed4d2fe04d9939b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
#importamos la libreria de transformers
import transformers
#y multiples clasificadores
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

In [ ]:
#Importamos la libreria para hacer el split train/test
from sklearn.model_selection import train_test_split

In [ ]:
# Importamos las librerias de pytorch
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:
# Bajamos los datos
!curl -O http://srodriguez.me/Datasets/imdb.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25.7M  100 25.7M    0     0  4347k      0  0:00:06  0:00:06 --:--:-- 5580k


In [ ]:
#Los descomprimimos
!unzip imdb.zip

Archive:  imdb.zip
  inflating: IMDB Dataset.csv        


In [ ]:
#Leemos el DataFrame
df = pd.read_csv("IMDB Dataset.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [ ]:
#mostramos los primeros 5
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
#Muestreamos 10.000 reviews al azar
#  Por temas de tiempo, ocupamos 30000
df = df.sample(30000)
df.reset_index(drop=True,inplace=True)

In [ ]:
# Transformamos el output en valores numericos
df['sentiment'] = df['sentiment'].map({'positive':1,'negative':0})

In [ ]:
#Mostramos la primer review
df.review[0]

'If you can stomach the campiness, this movie should make you laugh out loud several times. It did for me, at least. I\'ll only mention one of my favorite elements: the "underwater shots" of the "fish" "swimming". The sound which accompanies those shots is great too.<br /><br />One last note: William Katt is actually a pretty decent actor. I hope he gets another day in the sun; watching him as "The Greatest American Hero" was a fun part of my childhood a couple of decades ago, and he hasn\'t been very visible since. He seemed kind of depressed in this movie. Somebody give him a fun job on a good show, or something.'

In [ ]:
#Obtenemos set de train, validacion y testing
temp_df, test_df = train_test_split(df,test_size=0.3,random_state=42)
train_df, val_df = train_test_split(temp_df,test_size=0.1,random_state=42)

train_df.reset_index(drop=True,inplace=True)
val_df.reset_index(drop=True,inplace=True)
test_df.reset_index(drop=True,inplace=True)

In [ ]:
val_df

,review,sentiment
0,"This movie was awful! Ashley Rose Orr, while a...",0
1,I grew up watching and loving TNG. I just rece...,0
2,Omar Epps is an outstanding actor. I really th...,1
3,As always Joan Hickson is wonderful as Jane M....,1
4,"I grew up watching, and loving this cartoon ev...",1
...,...,...
2095,there are those movies that are bad they are f...,0
2096,"I love cheesy horror movies, I think dead aliv...",0
2097,One of the things that makes this Ealing comed...,1
2098,This is not a good film. The acting is remarka...,0


In [ ]:
lengths = train_df.review.apply(lambda x: len(x.split(" ")))
lengths.max()

2278

In [ ]:
#Cargamos un modelo de bert y su tokenizador
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
#Hagamos una prueba de como bert tokeniza una oración
tokenizer.tokenize("Hi, this is a test to see how Bert tokenizer processes a sentence")

['hi',
 ',',
 'this',
 'is',
 'a',
 'test',
 'to',
 'see',
 'how',
 'bert',
 'token',
 '##izer',
 'processes',
 'a',
 'sentence']

In [ ]:
#El vocabulario no es necesario ajustarlo, ya viene pre-entrenado y po lo mismo, 
#La transformación a indices númericos se utiliza con el metodo encode plus
tokenizer.encode_plus("Hi, this is a test to see how Bert tokenizer processes a sentence")

{'input_ids': [101, 7632, 1010, 2023, 2003, 1037, 3231, 2000, 2156, 2129, 14324, 19204, 17629, 6194, 1037, 6251, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
#Vamos a generar un CustomDataset (una formar de cargar datos en batches)
#Para pytorch, nos va a facilitar el entrenamiento
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.sentence = dataframe.review.values
        self.targets = self.data.sentiment.values
        self.max_len = max_len
    def __len__(self):
        return len(self.sentence)
    def __getitem__(self, index):
        sentence = str(self.sentence[index])
        sentence = " ".join(sentence.split())
        inputs = self.tokenizer.encode_plus(
            sentence,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids'] #Ids del vocabulario
        mask = inputs['attention_mask'] #Mascaras para definir donde la atencion deberia ver
        token_type_ids = inputs["token_type_ids"] #requerido pero no utilizado
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
#Generamos nuestra conexion a la GPU
from torch import cuda
device = 'cuda:0' if cuda.is_available() else 'cpu'

In [ ]:
device

'cuda:0'

In [ ]:
# Definimos constantes para la ejecucion de nuestro codigo
MAX_LEN = 200 # 2470
TRAIN_BATCH_SIZE = 8
TEST_BATCH_SIZE = 8
EPOCHS = 5
LEARNING_RATE = 1e-05

In [ ]:
#Transformamos los df en datasets
training_set = CustomDataset(train_df, tokenizer, MAX_LEN)
val_set = CustomDataset(val_df, tokenizer, MAX_LEN)
test_set = CustomDataset(test_df, tokenizer, MAX_LEN)

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

#Luego generamos los dataloaders de pytorch

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(test_set, **test_params)

In [ ]:
#Vamos a definir una clase de BERT modificada
class BERTClass(torch.nn.Module):
    def __init__(self,b_model):
        super(BERTClass, self).__init__()
        self.l1 = b_model # Definimos que la primera capa sea el modelo de bert que recibe por parametro
        self.l2 = torch.nn.Dropout(0.1) # A la salida de bert, vamos a aplicar un dropout de 0.1
        self.l3 = torch.nn.Linear(768, 1) #Finalmente vamos a definir una capa densa (combinación lineal) de la dimension de bert a nuestra neurona de salida
        self.out_activation = torch.nn.Sigmoid() # función de activación (que no vamos a utilizar)
    
    def forward(self, ids, mask, token_type_ids):
        #Hacemos el forward pass por BERT, dropout y la capa densa
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids) 
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        #output = self.out_activation(output)
        return output

#Generamos un modelo de BERTClass
model = BERTClass(bert_model)
#llevamos el modelo a la GPU
model.to(device)

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [ ]:
#Definimos nuestra funcion de perdida
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
#El optimizador
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
#Definimos nuestro loop de entrenamiento
def train(epoch):
    
    model.train()
    loss_acum=0
    for iters , data in tqdm(enumerate(training_loader, 0), total = len(training_loader)):
        #print(len(data['ids'][0]))
        #print(len(data['mask'][0]))
        #print(len(data['token_type_ids'][0]))
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        #print(len(ids),len(mask),len(token_type_ids))
        #print(ids,mask,token_type_ids,targets)
        outputs = model(ids, mask, token_type_ids)
        outputs = outputs.squeeze(1)
        #print(outputs,targets)
        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        loss_acum += loss.item()
        if iters%5000==0:
            print(outputs,targets)
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(iters)
    return loss_acum/iters

In [ ]:
#Definimos nuestro loop de validacion/testing
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0),total = len(testing_loader)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)

            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).squeeze(1).cpu().detach().numpy())
    
    return np.array(fin_outputs), np.array(fin_targets)

In [ ]:
#iteramos por nuestras epocas
for epoch in range(EPOCHS):
    epoch_loss = train(epoch)
    print(f'epoch loss: {epoch_loss}')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


tensor([-0.5551, -0.5339, -0.5503, -0.5900, -0.7180, -0.6057, -0.5718, -0.6504],
       device='cuda:0', grad_fn=<SqueezeBackward1>) tensor([0., 0., 0., 1., 0., 1., 1., 1.], device='cuda:0')
Epoch: 0, Loss:  0.7411922216415405

2362
epoch loss: 0.3071510972952743


tensor([ 2.0053,  2.1685,  3.0031,  4.1094, -5.2547,  3.5835,  1.1124, -4.8856],
       device='cuda:0', grad_fn=<SqueezeBackward1>) tensor([1., 1., 1., 1., 0., 1., 0., 0.], device='cuda:0')
Epoch: 1, Loss:  0.21700528264045715

2362
epoch loss: 0.17272398194020566


tensor([ 1.7591,  4.0843,  4.8774, -3.6525,  4.7223,  3.7324,  2.4770, -5.6787],
       device='cuda:0', grad_fn=<SqueezeBackward1>) tensor([1., 1., 1., 0., 1., 1., 1., 0.], device='cuda:0')
Epoch: 2, Loss:  0.040666934102773666


KeyboardInterrupt: ignored

In [ ]:
from sklearn import metrics
import numpy as np

#Corremos un loop de testing
for epoch in range(1):
    outputs, targets = validation(epoch)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
#Calculamos los valores binarios y luego obtenemos las metricas de desempeño
outputs_bin = (outputs >= 0.5)
accuracy = metrics.accuracy_score(targets, outputs_bin)
f1_score_micro = metrics.f1_score(targets, outputs_bin, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs_bin, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

Accuracy Score = 0.9077777777777778
F1 Score (Micro) = 0.9077777777777778
F1 Score (Macro) = 0.9077579354847887


In [ ]:
print(metrics.classification_report(targets, outputs_bin))

              precision    recall  f1-score   support

         0.0       0.91      0.90      0.91      4471
         1.0       0.90      0.92      0.91      4529

    accuracy                           0.91      9000
   macro avg       0.91      0.91      0.91      9000
weighted avg       0.91      0.91      0.91      9000



In [ ]:

test_string = "i hated the movie, it did not have very beautiful scenes"

converted_string = tokenizer.encode_plus(test_string, return_tensors='pt')

converted_string

{'input_ids': tensor([[ 101, 1045, 6283, 1996, 3185, 1010, 2009, 2106, 2025, 2031, 2200, 3376,
         5019,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
inputs_ids = (converted_string['input_ids']).to(device, dtype = torch.long)
attention_mask = (converted_string['attention_mask']).to(device, dtype = torch.long)
token_type_ids = (converted_string['token_type_ids']).to(device, dtype = torch.long)

outputs = model(inputs_ids,attention_mask, token_type_ids)
torch.sigmoid(outputs).squeeze(1).cpu().detach().numpy()

array([0.00333876], dtype=float32)